In [1]:
import pandas as pd
import os

def set_column_values(df):
    # Ensure the DataFrame is a pandas DataFrame
    if not isinstance(df, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame")
    
    # Set column values
    df["pelvic_tx"] = 0
    df["pelvic_ty"] = 0
    df["pelvic_tz"] = 0

    return df

def trim_dataframe(df, column_name, x, y):
    # Ensure the DataFrame is a pandas DataFrame
    if not isinstance(df, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame")
    
    # Find the index of the maximum absolute value in the column
    max_index = df[column_name].abs().idxmax()

    # Convert the index to integer, if it isn't already
    if df.index.dtype != int:
        # Create a new DataFrame with integer index
        df = df.reset_index(drop=True)

        # Recalculate the max_index in terms of the integer index
        max_index = df[column_name].abs().idxmax()
    
    # Calculate the start and end indices
    start_index = max(0, max_index - x)  # Ensure the start index is not less than 0
    end_index = min(df.shape[0], max_index + y + 1)  # Ensure the end index is not more than the number of rows

    # Slice the DataFrame
    trimmed_df = df.iloc[start_index:end_index]

    return trimmed_df

def prepare_opensim_file(file_path, re_reference = True, trim = False, trimLeft = 120, trimRight = 120, trimVariable = 'knee_angle_r',  write_csv=False):
    with open(file_path, 'r') as file:
        for i, line in enumerate(file):
            if 'endheader' in line:
                skiprows = i + 1
                break
    data = pd.read_csv(file_path, delimiter='\t', skiprows=skiprows)
    #data.set_index('time', inplace=True)
    
    if (re_reference):
        data = set_column_values(data)
        
    if (trim):
        data = trim_dataframe(data, trimVariable, trimLeft, trimRight)

    if write_csv:
        csv_file_path = os.path.splitext(file_path)[0] + '.csv'
        data.to_csv(csv_file_path, index=False)
        
    return data

def write_opensim_file(data, file_path, csv_file_path=None):
    if csv_file_path is not None:
        data = pd.read_csv(csv_file_path)
    
    nRows = len(data)
    nColumns = len(data.columns)

    header = f"""Coordinates
    version=1
    nRows={nRows}
    nColumns={nColumns}
    inDegrees=yes

    Units are S.I. units (second, meters, Newtons, ...)
    If the header above contains a line with 'inDegrees', this indicates whether rotational values are in degrees (yes) or radians (no).

    endheader
    """

    with open(file_path, 'w') as file:
        file.write(header)
    data.to_csv(file_path, sep='\t', mode='a', index=False, float_format='%.8f')
    
    

In [2]:
# Read OpenSim file and write out a .csv with data re-referenced to the pelvis position
data = prepare_opensim_file('Data/box1_enable_rotated_Kinematics_q.sto', re_reference = True, trim = True, trimLeft = 360, trimRight = 200, write_csv=True)

In [3]:
# Write OpenSim file
#write_opensim_file(data, 'newfile.sto')

# Write data from .csv file to OpenSim file
#write_opensim_file(None, 'box1_enable_rotated_Kinematics_q.sto', csv_file_path='Data/box1_enable_rotated_Kinematics_q.csv')